# Как передать функцию в функцию

В Ruby есть блоки. Блок кода - это просто кусок кода, обрамленный ключевыми словами `do` и `end` или в фигурных скобках.

## Немного теории

**Блок - это не объект класса, блок - это специальная конструкция языка**

Именно поэтому работа с блоками - это **специальный встроенный в Ruby механизм**.

Итак, зачем же действительно нужны блоки?

## Любой функции можно передать блок кода. Вызвать этот блок можно ключевым словом yield

Это первый и самый основной способ передачи функции в функцию. Это даже не совсем передача функции, скорее передача куска кода, который умеет принимать параметры (а может быть и без них) и что-то с ними делать.

На уровне языка каждая функция умеет принимать **последним** неявным параметром блок кода. 

**Ключевое слово yield работает только с блоком**

In [12]:
def foo(x)
  yield x
end

foo(1) do |x|
  x + 1
end

2

Конструкция `yield x` значит примерно следующее: "попытайся вызвать исполнение блока кода, переданного функции, и дай ему в качестве аргументов все то, что стоит после `yield`" (как вы помните, скобки ставить не обязательно, а если бы они стояли, то это выглядело бы как `yield(x)`).

**Примечание**

Бывают однострочные блоки, в данном случае он бы выглядел так (это не код, а текст):
```ruby
foo(1) {|x| x + 1 }
```

Можно так писать для экономии места.

Таким образом, конструкция
```ruby
[1, 2, 3].map {|x| x**2 }
```
**это и есть передача блока методу map, который внутри себя делает yield!**

yield - достаточно тупая инструкция

In [13]:
foo(2)

LocalJumpError: no block given (yield)

Зато если напутать с количеством параметров, которые умеет принимать блок, ничего криминального не случится.

In [16]:
foo(1) do
  p 'У меня нет параметров'
end

"У меня нет параметров"


"У меня нет параметров"

**Что же сделать, чтобы не упало?**

Проверить наличие блока:

In [17]:
def smart_foo(x)
  if block_given?
    yield x
  else
    p 'Блок не передан'
    x
  end
end

smart_foo(1) do |x|
  p 'Блок передан'
  x + 1
end

smart_foo(1)

"Блок передан"
"Блок не передан"


1

## Функцию в чистом виде передать нельзя

В языке Python функция - это такой же идентификатор переменной. Этому идентификатору можно присвоить все, что угодно, хоть число. Ничего за это не будет.

```python
def foo(x):
    return x + 1

foo = 12  # Все ок.

# Благодаря этому можно писать так

def foo(x):
    return x + 1

def caller(argument, function):
    return function(argument)
    
caller(10, foo)  # foo - это объект класса, у которого есть операция (). Поэтому он и ведет себя как функция, и его можно передавать в другую функцию как переменную, противоречий нет. 
```

В Ruby функция - это **особая конструкция языка**, как, в принципе, и в С, С++, Pascal и многих других языках. Пример выше тут уже не прокатит. Но что делать, если я не хочу каждый раз писать один и тот же блок кода, а присвоить блок кода переменной нельзя?

**Выход есть всегда :)**

## Можно передать анонимную функцию - lambda

Вы сделаете то же самое, что и в Python: поставите в соответствие переменной функцию, которая будет вызываться каждый раз без дублирования кода. 

Есть два варианта синтаксиса:

In [82]:
lambda_function1 = ->(x) { x + 1 }
lambda_function2 = lambda {|x| x + 1 }

p lambda_function1.call 1
lambda_function2.call 1

2


2

Можно писать и так, но это просто многострочные версии тех же блоков, в дальнейшем заострять на этом внимание мы не будем.

In [30]:
lambda_function1 = ->(x) do 
  x + 1
end

lambda_function2 = lambda do |x| 
  x + 1
end

#<Proc:0x00000002b7e470@<main>:4 (lambda)>

При желании можно сразу сконструировать лямбду при вызове функции или передать переменную, содержащую ссылкку на нее:

In [32]:
def foo(x, l)
  l.call x
end

p foo(1, lambda_function2)
foo(1, ->(x) {x + 1})

2


2

**Примечание**

Лямбды придумали как раз для таких конструкций, поэтому их еще называют анонимными функциями. Вам не всегда удобно объявлять функцию где-то наверху, чтобы потом **один** раз ее использовать. Для этого и придумаля лямбды.

## Можно передать специальный объект-обертку над функцией - Proc (от слова Procedure)

Так же как и lambda, Proc умеет себя вызывать, у него есть метод call. Конструируется он тоже с помощью блока, который передается в его конструктор или в конструкцию с ключевым словом `proc`:

In [38]:
def foo(x, f)
  f.call x
end

# Первый способ - через конструктор с блоком.
function1 = Proc.new do |x|
  x + 2
end

# Второй способ - через специальную конструкцию языка.
function2 = proc {|x|  x + 2}

p foo(1, function1)
p foo(1, function2)

3
3


3

### И еще: Symbol умеет преобразовываться в Proc

In [56]:
def function(x)
  x**2
end

f = :function.to_proc

#<Proc:0x000000029e5118(&:function)>

Однако вызвать такой прок нельзя из-за особенностей реализации функций.

In [57]:
f.call 2

NoMethodError: private method `function' called for 2:Fixnum

Для решения этой проблемы придумали класс Method

In [58]:
method(:function)

#<Method: Object#function>

In [83]:
method(:function).to_proc.call 2

4

Вызов #to_proc в данном случае можно опустить, и так сработает.

### А для чего это вообще придумали?

Чтобы писать вместо

In [60]:
['a', 'b', 'c'].map {|x| x.upcase }

["A", "B", "C"]

In [62]:
['a', 'b', 'c'].map &:upcase

["A", "B", "C"]

### Что такое &

В данном случае (то есть при передаче параметров в функцию) это указание интерпретатору **преобразовать Symbol, который содержит имя функции, в Proc**.

Преобразование не сумеет сделать всего:

In [63]:
def foo(x)
  x * 3
end

[1, 2, 3].map &:foo

NoMethodError: private method `foo' called for 1:Fixnum
Did you mean?  floor

Поэтому для **своих функций** нужно писать

In [64]:
[1, 2, 3].map &method(:foo)

[3, 6, 9]

Как вы помните, любой метод неявно принимает блок кода. Однако можно явно указать Ruby, что мы хотим принять последний (именованный нами параметр) как блок кода. Для этого нужно поставить & перед ним.

In [86]:
def foo(arg, &block)
  block.call arg if block_given? or nil
end

def block arg
  arg * 2
end

foo 10, &method(:block)

20

In [89]:
p foo 20

nil


Интересная задачка - объяснить, как это работает

In [90]:
def blah(&block)
  yadda(block)
end
 
def yadda(block)
  foo(&block)
end
 
def foo(&block)
  block.call
end
 
blah do
  puts "hello"
end

hello


Ответ [тут](https://www.skorks.com/2013/04/ruby-ampersand-parameter-demystified/)

## Appendix 1 - Отличия lambda и proc

Формально lambda и Proc - объекты одного класса.

In [65]:
l = ->() { p 'lambda' }
pr = proc { p 'proc' }
  
p l.class
p pr.class

Proc
Proc


Proc

Однако различия все же есть, для этого даже добавили специальный метод

In [70]:
p l.lambda?
pr.lambda?

true


false

А в чем же отличие?

### Передача параметров

Если лямбда вызывается с большим, или меньшим количеством аргументов, чем необходимо, тогда Ruby выдает ошибку ArgumentError.

Однако когда Proc вызывается с большим количеством аргументов, чем необходимо, никакой ошибки не возвращается и лишние аргументы просто отбрасываются. Когда процедура вызывается с меньшим количеством аргументов, то те параметры, которые не получили необходимых значений, приобретают значение nil.

In [73]:
l.call 10, 11

ArgumentError: wrong number of arguments (given 2, expected 0)

In [74]:
pr.call 10, 11

"proc"


"proc"

In [76]:
new_pr = proc {|x, y| p x, y }
new_pr.call 1

1
nil


[1, nil]

### Возврат значений

Lambda - все ок

In [79]:
def foo
  l = ->(x) { return x }
  x = l.call 1
  "Я возвращаю X из foo: #{x}"
end

foo

"Я возвращаю X из foo 1"

Proc - WTF?!

In [80]:
def foo
  pr = proc {|x| return x }
  x = pr.call 1
  "Я возвращаю X из foo: #{x}"
end

foo

1

Только что на ваших глазах Proc прервал выполнение функции foo! Как ни странно, если убрать return, все будет работать ок.

In [81]:
def foo
  pr = proc {|x| x }
  x = pr.call 1
  "Я возвращаю X из foo: #{x}"
end

foo

"Я возвращаю X из foo: 1"

### Если копнуть глубже

То можно почитать [это](https://stackoverflow.com/questions/1435743/why-does-explicit-return-make-a-difference-in-a-proc)